# <center><h1> ⛓ CHAINS 🔗</h1></center>



### Оглавление ноутбука
<img src='https://i.ytimg.com/vi/W3AoeMrg27o/maxresdefault.jpg' align="right" width="600" height="650" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🚀 Введение </a></li>
<li><a href="#c2">💊 Generic Chains </a></li><ul type="square">
<li><a href="#c3">LLMChain 🔗 </a>
<li><a href="#c4">TransformChain 🤖 </a>
<li><a href="#c5">SequentialChain 🔗➕⛓</a>
<li><a href="#c6">Router Chain 🈯️ ➡️ 🇯🇵</a></ul>
<li><a href="#c7">🛠 Utility Chains 🪝 </a></li>
<li><a href="#c8">🧸 Выводы и заключения ✅ </a></li>
    
</ul></font></p>

<div class="alert alert-info">
    
Как вы могли догадаться из названия, **цепочки (Chains)** - это один из фундаментальных строительных блоков фрэймворка `LangChain`. Они представляют собой просто цепочку компонентов, которые будут выполнены в определённом порядке. <br>

*Официальным определением цепочек является следующее:* <br>
Цепочка состоит из звеньев, которые могут быть как примитивами, так и другими цепочками. Примитивами могут быть либо промпты, LLM, utils, либо другие цепочки. Таким образом, цепочка — это, по сути, конвейер, который обрабатывает входные данные, используя определенную комбинацию примитивов. Интуитивно это можно рассматривать как «шаг», который выполняет определенный набор операций над входными данными и возвращает результат. Это может быть что угодно: от прохождения через LLM на основе промпта до применения Python-функции к тексту.

Цепочки делятся на несколько основных типов: 
* `Generic chains` - общего назначения
* `Utility chains` - выполняющие конкретную функцию (например, вычислять математические выражения)
* `Combine Documents chains` - цепочки для работы с документами <br>
и другие.

В этом ноутбуке остановимся на первых двух типах, третий специфический тип мы рассмотрим позднее.

# <center> 💊 Generic Chains - цепочки общего назначения </center> 

<div class="alert alert-info">
    
В `LangChain` есть 3 основных типа `Generic Chains`:
* `LLM Chain` - базовый тип.
* `Transform Chain` - позволяет преобразовывать текст и применять к нему различные функции.
* `Sequentional Chain` - позволяет собирать последовательности из нескольких цепочек.

*Попробуем рассмотреть их всех на одном примере* 🤪

## LLMChain 🔗

Самой простой из этих цепочек является `LLMChain`. С ней мы уже знакомились в предыдущих уроках.

In [81]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [82]:
#!pip install langchain openai tiktoken -q

In [83]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [84]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain.chat_models import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [85]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI


#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите ваш OpenAI API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш OpenAI API ключ ········


<div class="alert alert-info">
Создадим цепочку для переписывания текста в различных стилях.

In [86]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# создадим шаблон и промпт
template = ''' Перепиши этот текcт: {output_text}
В стиле {style}.
Результат:'''

prompt = PromptTemplate(input_variables=['output_text', 'style'], template=template) 

# создаём цепочку
style_changer_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

In [88]:
text = '''Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей, 
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.'''
style = 'Роман 18 века'

style_changer_chain.run({'output_text': text, 'style': style})

'Привечаемъ въ курсъ по тюнингу языковыхъ моделей! \nТутъ научимъ васъ, какъ бы измѣнити модельки з еще болѣе вѣличественными и растолковать ихъ потенциалъ до максимума. \nДадимъ вамъ совѣти по самымъ прямъновременийнымъ способамъ обучения и управления языка, \nа тождѣ научимъ васъ практическимъ навыкамъ для издѣкованія трудовыхъ задачъ в обработкѣ нежурная языка.'

Видим, что цепочка с моделью неплохо справилась с задачей.

## <center> TransformChain 🤖

<div class="alert alert-info">
    
Теперь давайте представим, что мы получаем откуда-то сырой необработанный текст, который содержит пустые строки и много ненужных пробелов. Как вы помните, нам приходится считать количество израсходованных токенов, и не хочется их дополнительно тратить на ненужные лишние символы. К тому же такой текст выглядит неопрятно. <br>
Давайте создадим цепочку, которая будет очищать поданный в неё текст от ненужных символов. <br>
В этом нам поможет `Transform Chain`.


Для начала создадим функцию, которая будет убирать лишние пробелы и строки в исходном тексте.

In [15]:
import re

def del_spaces(inputs: dict) -> dict:
    text = inputs["text"]
    
    # заменяем пустые строки и дополнительные пробелы на один, используя регулярные выражения
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [33]:
# создаём преобразующую цепочку
from langchain.chains import TransformChain

text_clean_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=del_spaces)

<div class="alert alert-info">
    
**Обратите внимание**, что при создании этой цепочки, мы не подаём llm как аргумент. Можно подумать, что отсутствие LLM делает такую цепочку более слабой по сравнению с другими, но далее мы увидим, что комбинируя её с другими цепочками, можно добиться хороших результатов и сэкономить токены.<br>
Посмотрим как она работает:

In [19]:
# Возьмём наш предыдущий текст и "загрязним его"
dirty_text = '''Приветствуем на курсе по        тюнингу языковых моделей     ! 

Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по   самым свежим методам  обучения и настройки языковых                    моделей,

а также научим практическим навыкам для                     решения сложных задач в обработке естественного языка.'''

In [20]:
# Запустим, и видим, что цепочка задачу выполнила
text_clean_chain.run(dirty_text)

'Приветствуем на курсе по тюнингу языковых моделей ! \nТут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. \nМы дадим вам советы по самым свежим методам обучения и настройки языковых моделей,\nа также научим практическим навыкам для решения сложных задач в обработке естественного языка.'

<div class="alert alert-info">
    
Как вы уже наверное догадались (по названиям входных и выходных переменных), мы собираемся подать выход из `Transform Chain` в `LLMChain`. Чтобы их объединить и использовать как одну интегрированную цепочку, воспользуемся `Sequential Chain`.

## <center>  SequentialChain 🔗➕⛓ - последовательные цепочки

<div class="alert alert-info">
    
Два основных типа `Sequential Chain`:
* `SimpleSequentialChain`: простейшая форма последовательной цепочки, где каждый шаг имеет один вход/выход, а выход одного шага является входом следующего.
* `SequentialChain`: более общая форма последовательной цепочки, допускающая несколько входов/выходов.

Воспользуемся вторым вариантом:

In [29]:
from langchain.chains import SequentialChain

In [42]:
sequential_chain = SequentialChain(chains=[text_clean_chain, style_changer_chain],
                                   input_variables=['text', 'style'],
                                   output_variables=['final_output'])

In [45]:
style = 'Rap'
print(sequential_chain.run({'text': dirty_text, 'style': style}))

Привет, друзья, на курсе мы вас ждем,
Тюнинг языковых моделей – это наш домен.
Нашей целью станет модели улучшить,
Их потенциал раскрыть и вдохновить.

Свежие методы обучения мы расскажем,
И настройку языковых моделей научим.
Практические навыки дадим вам тоже,
Для решения сложных задач успешно.

Приготовьтесь, вас ждет удивительный путь,
Где обработка естественного языка в руках у вас – в шутку.
Будем стилем Рэпа искренне вдохновлять,
И вместе этот курс максимум раскрыть.


Так же к `Generic Chains` можно отнести семейство `Router Chains`,  создающее цепочки, которые динамически выбирают наиболее подходящую следующую цепь в зависимости от входных данных.

## <center> Router Chain 🈯️ ➡️ 🇯🇵 - определитель тем 

<div class="alert alert-info">
    
Например, если у вас есть две цепочки с LLM хорошо натренированными под разные задачи, одна хорошо разбирается в цветах, а другая в футболе. То когда вы зададите вопрос, `Router Chain` может определить тему и отправить запрос в соответствующую цепочку. <br>
Рассмотрим на примере:

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

Так как у нас нет 2-х разных предобученых моделей под конкретную задачу, попросим ChatGPT отвечать нам, как-будто она специалист в 2 различных областях, с помощью промптов. 

In [63]:
florist_template = """Ты очень опытный флорист и ботаник, знаешь всё о цветах, растениях.
Тебе нравится отвечать на вопросы о том, как выбирать и ухаживать за растениями. 
Ты отвечаешь так, что всё становится ясно даже начинающему цветоводу. 
Вот вопрос:
{input}"""

football_template = """Ты спортивный журналист с большим опытом, твоя основная специализация футбол.
Ты знаешь всё о футбольных командах и игроках, и очень любишь отвечать на вопросы о футболе, но кратко и по делу.
Вот вопрос:
{input}"""

In [64]:
# Создаём словари с описанием моделей
prompt_infos = [
    {
        "name": "florist",
        "description": "Отлично отвечает на вопросы о цветах и растениях",
        "prompt_template": florist_template,
    },
    {
        "name": "football",
        "description": "Хорошо отвечает на вопросы о футболе",
        "prompt_template": football_template,
    },
]

In [65]:
# Создаём в цикле 2 LLMChain под разные задачи
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Создаём дефолтную цепочку, которая будет поддерживать диалог
default_chain = ConversationChain(llm=llm, output_key="text")

Чтобы `Router Chain` могла работать ей нужен специальный `router promnt` на основе специального шаблона `MULTI_PROMPT_ROUTER_TEMPLATE`. <br>
Посмотрим на него:

In [66]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [67]:
# готовим описание наших моделей для специального промпта
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# создаём шаблон и промпт
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# создаём RouterChain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [68]:
#print(router_template)

И теперь, чтобы магия заработала, объединяем все созданные цепочки в одну `MultiPromptChain`.

In [69]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [70]:
print(chain.run("Какие цвета у Барселоны?"))



> Entering new MultiPromptChain chain...
football: {'input': 'Какие цвета у Барселоны?'}
> Finished chain.
Цвета Барселоны являются сине-гранатовыми, а точнее, клуб использует гранатовый (темно-красный) цвет в сочетании с синим.


In [62]:
print(chain.run("Сколько раз в неделю поливать кактус?"))



> Entering new MultiPromptChain chain...
florist: {'input': 'Сколько раз в неделю поливать кактус?'}
> Finished chain.
Кактусы имеют особенности водопотребления, и частота полива может варьироваться в зависимости от условий окружающей среды и времени года. В целом, для большинства видов кактусов рекомендуется поливать раз в две-три недели. Однако, важно учитывать такие факторы, как тип горшка и почва, размер и возраст растения, а также количество света и температура в помещении. Регулярно проверяйте влажность почвы с помощью влагомера или просто проводя палец по поверхности грунта. Если верхний слой почвы высох, это сигнал для полива. Однако, помните, что переувлажнение может привести к гниению корней, поэтому важно не залить растение. Вывод: поливая кактус, следите за индивидуальными потребностями растения и адаптируйте частоту полива в соответствии с ними.


In [71]:
print(chain.run("Какие цветы у Барселоны?"))



> Entering new MultiPromptChain chain...
football: {'input': 'Какие цветы у Барселоны?'}
> Finished chain.
Официальные цвета Футбольного клуба Барселона - гранатовый и голубой.


# <center> 🛠 Utility Chains 🪝 </center>


<div class="alert alert-info">
    
Судя по названию, `Utility Chains` выполняют какую-либо утилитарную функцию: математические расчеты, отправляют запросы в интернет и работают с ответом запроса, выполняют bash-скрипты и.т.п. <br>
Их достаточно много разных, рассмотрим пример с математической цепочкой:

In [72]:
from langchain.chains import LLMMathChain

math_chain = LLMMathChain(llm=llm, verbose=True)

In [74]:
print(math_chain.run('Сколько будет 25 возвести в степень 0.25'))



> Entering new LLMMathChain chain...
Сколько будет 25 возвести в степень 0.25```text
25**0.25
```
...numexpr.evaluate("25**0.25")...

Answer: 2.23606797749979
> Finished chain.
Answer: 2.23606797749979


<div class="alert alert-info">
    
Видим, цепочка получила вопрос на естественном языке и отправила его в модель. LLM вернул код Python, который скомпилировала цепочка, чтобы дать нам ответ. <br>
Возникает вопрос: Как LLM узнала, что мы хотим, чтобы он возвращал код Python? <br>
Ответ мы можем обнаружить, если посмотрим на промпт этой цепочки:

In [76]:
print(math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



<div class="alert alert-info">
    
Мы буквально говорим LLM, что для решения сложных математических задач он не должен пытаться решать математические задачи самостоятельно, а вместо этого должен распечатать код Python, который выполнит расчет математической задачи. Вероятно, если бы мы просто отправили запрос без какого-либо контекста, LLM попыталась бы (и не смогла) вычислить это самостоятельно. <br>
Это можно проверить... давайте попробуем! 🧐

In [77]:
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.run('Сколько будет 25 возвести в степень 0.25'))

25 возвести в степень 0.25 равняется приблизительно 1.6818.


Неверный ответ! В этом и заключается сила промптинга. <br>
Вывод: разумно используя промптинг, мы можем заставить LLM избегать распространенных ошибок, явно программируя его на определенное поведение.

# <center id="part6"> 🧸 Выводы и заключения ✅: <br>

<div class="alert alert-info">
    
Во всех, приведенных выше, примерах вы могли заметить, что цепочка в своей основе имеет тот или иной специально написанный промпт, т.е. по сути можно было бы обходиться без них, проектируя новый сложный промпт под каждую задачу. Но согласитесь, что с помощью цепочек `LangChain` это делать гораздо проще и удобнее. Особенно, если мы говорим про крупные сервисы. Т.е. цепочки позволяют нам больше сосредоточиться на продумывании логики и функционала сервиса, а не на дизайне промптов.